### Initialize and Login

In [1]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from datetime import datetime
import calendar
from tqdm import tqdm

# For Charts
import plotly.graph_objects as go

# To access .env file
from dotenv import load_dotenv

import os
import time
import talib
from talib import abstract

In [2]:
# Access .env file
load_dotenv(".env")

# Access variables directly
username = int(os.getenv("USER_NAME"))
password = os.getenv("PASSWORD")
server = os.getenv("SERVER")

symbols = os.getenv("SYMBOLS")

In [3]:
# Initialize and log in to MetaTrader 5
# connect to MetaTrader 5
try:
    # Initialize MetaTrader 5
    if mt5.initialize():
        print("MetaTrader successfully initialized")
        
        # Log in to MetaTrader 5
        if mt5.login(login=username, password=password, server=server):
            print("MetaTrader login successful")
                
        else:
            print("MetaTrader login failed")
            mt5.shutdown()  # Ensure shutdown if login fails
    else:
        print("Failed to initialize MetaTrader 5")
except Exception as e:
    print(f"An error occurred: {e}")

MetaTrader successfully initialized
MetaTrader login successful


### TA-Library

In [31]:
list(talib.get_function_groups().keys())

['Cycle Indicators',
 'Math Operators',
 'Math Transform',
 'Momentum Indicators',
 'Overlap Studies',
 'Pattern Recognition',
 'Price Transform',
 'Statistic Functions',
 'Volatility Indicators',
 'Volume Indicators']

In [32]:
talib.get_function_groups()['Momentum Indicators']

['ADX',
 'ADXR',
 'APO',
 'AROON',
 'AROONOSC',
 'BOP',
 'CCI',
 'CMO',
 'DX',
 'MACD',
 'MACDEXT',
 'MACDFIX',
 'MFI',
 'MINUS_DI',
 'MINUS_DM',
 'MOM',
 'PLUS_DI',
 'PLUS_DM',
 'PPO',
 'ROC',
 'ROCP',
 'ROCR',
 'ROCR100',
 'RSI',
 'STOCH',
 'STOCHF',
 'STOCHRSI',
 'TRIX',
 'ULTOSC',
 'WILLR']

In [ ]:
talib.get_functions()

['HT_DCPERIOD',
 'HT_DCPHASE',
 'HT_PHASOR',
 'HT_SINE',
 'HT_TRENDMODE',
 'ADD',
 'DIV',
 'MAX',
 'MAXINDEX',
 'MIN',
 'MININDEX',
 'MINMAX',
 'MINMAXINDEX',
 'MULT',
 'SUB',
 'SUM',
 'ACOS',
 'ASIN',
 'ATAN',
 'CEIL',
 'COS',
 'COSH',
 'EXP',
 'FLOOR',
 'LN',
 'LOG10',
 'SIN',
 'SINH',
 'SQRT',
 'TAN',
 'TANH',
 'ADX',
 'ADXR',
 'APO',
 'AROON',
 'AROONOSC',
 'BOP',
 'CCI',
 'CMO',
 'DX',
 'MACD',
 'MACDEXT',
 'MACDFIX',
 'MFI',
 'MINUS_DI',
 'MINUS_DM',
 'MOM',
 'PLUS_DI',
 'PLUS_DM',
 'PPO',
 'ROC',
 'ROCP',
 'ROCR',
 'ROCR100',
 'RSI',
 'STOCH',
 'STOCHF',
 'STOCHRSI',
 'TRIX',
 'ULTOSC',
 'WILLR',
 'BBANDS',
 'DEMA',
 'EMA',
 'HT_TRENDLINE',
 'KAMA',
 'MA',
 'MAMA',
 'MAVP',
 'MIDPOINT',
 'MIDPRICE',
 'SAR',
 'SAREXT',
 'SMA',
 'T3',
 'TEMA',
 'TRIMA',
 'WMA',
 'CDL2CROWS',
 'CDL3BLACKCROWS',
 'CDL3INSIDE',
 'CDL3LINESTRIKE',
 'CDL3OUTSIDE',
 'CDL3STARSINSOUTH',
 'CDL3WHITESOLDIERS',
 'CDLABANDONEDBABY',
 'CDLADVANCEBLOCK',
 'CDLBELTHOLD',
 'CDLBREAKAWAY',
 'CDLCLOSINGMARUBOZU',


### Backtesting Optimized

#### 1 (Ongoing)
This code has the necessary modifications and is used to further create functions

In [171]:
# Choosing currency pair
symbols = "EURUSDm"

# Retrieve currency data in 1 minute timeframe with a time range
ticks = mt5.copy_rates_range(symbols, mt5.TIMEFRAME_M5, datetime(2024,3,1), datetime(2024,3,30))
data = pd.DataFrame(ticks)
data['time'] = pd.to_datetime(data['time'], unit='s')  # convert time including seconds
data.rename(columns={"tick_volume": "volume"}, inplace=True)

# Get the "point" or tick of the pair for symbol presumably = 0.00001
sym_point = mt5.symbol_info(symbols).point

# Define pip values and spread adjustment, a point is 0.00001
take_profit_pips = 0.0005  # 5 pips for take profit
stop_loss_pips = 0.0001    # 1 pips for stop loss

# Compute half spread as a temporary column
data['half_spread'] = data['spread']*sym_point / 2

# Sample DataFrame with close prices and moving averages (data)
# Calculate Moving Averages
data['MA_50'] = talib.SMA(data['close'], timeperiod=50)
data['MA_200'] = talib.SMA(data['close'], timeperiod=200)

# Initialize columns for trade entry prices, targets, and results
data['Entry_Price'] = None
data['Take_Profit'] = None
data['Stop_Loss'] = None
data['Trade'] = None  # 'Buy' or 'Sell'
data['Trade_Success'] = None
data['Pips'] = None

# Identify crossovers and calculate thresholds in one pass
buy_signals = (data['MA_50'] > data['MA_200']) & (data['MA_50'].shift(1) <= data['MA_200'].shift(1))
sell_signals = (data['MA_50'] < data['MA_200']) & (data['MA_50'].shift(1) >= data['MA_200'].shift(1))

# Buy signal: entry price + half spread, set TP and SL
data.loc[buy_signals, 'Entry_Price'] = data['close'] + data['half_spread']
data.loc[buy_signals, 'Take_Profit'] = data['Entry_Price'] + take_profit_pips
data.loc[buy_signals, 'Stop_Loss'] = data['Entry_Price'] - stop_loss_pips
data.loc[buy_signals, 'Trade'] = 'Buy'

# Sell signal: entry price - half spread, set TP and SL
data.loc[sell_signals, 'Entry_Price'] = data['close'] - data['half_spread']
data.loc[sell_signals, 'Take_Profit'] = data['Entry_Price'] - take_profit_pips
data.loc[sell_signals, 'Stop_Loss'] = data['Entry_Price'] + stop_loss_pips
data.loc[sell_signals, 'Trade'] = 'Sell'

# Check if TP or SL is hit in the next 1 to 200 periods for each trade row
for i, row in data.iterrows():
    if row['Trade'] in ['Buy', 'Sell']:
        trade_success = 'No Result'
        pips_result = 0  # Default pip result if no TP or SL is hit

        # Iterate through increasing periods (1 to 200) to check TP/SL
        for period in range(1, 1000):
            # Define the future high and low within the current period
            future_high = data['high'][i:i+period].max()
            future_low = data['low'][i:i+period].min()

            # Set conditions based on trade type
            if row['Trade'] == 'Buy':
                take_profit_hit = future_high >= row['Take_Profit']
                stop_loss_hit = future_low <= row['Stop_Loss']
            else:  # Sell
                take_profit_hit = future_low <= row['Take_Profit']
                stop_loss_hit = future_high >= row['Stop_Loss']

            # Determine if TP or SL is hit within this period
            if take_profit_hit:
                trade_success = 'Success'
                pips_result = take_profit_pips
                break  # Exit the loop once TP is hit
            elif stop_loss_hit:
                trade_success = 'Fail'
                pips_result = -1 * stop_loss_pips
                break  # Exit the loop once SL is hit

        # Update the DataFrame with the trade result
        data.at[i, 'Trade_Success'] = trade_success
        data.at[i, 'Pips'] = pips_result*10000

# Filter and view only the trades
trades_df = data.dropna(subset=['Trade'])

# Get the month name
month_name = datetime(2024,3,1).strftime("%b")

# Extract counts for each category
buy_sell = trades_df["Trade"].value_counts()
success_fail = trades_df["Trade_Success"].value_counts()
total_trades = trades_df.shape[0]   # For the total count of trades
sum_pips = trades_df.groupby("Trade_Success")["Pips"].sum().sum() # Sum of pips

# Combine values into a DataFrame with the desired structure
combined_df = pd.DataFrame({
    month_name: [
        total_trades,
        buy_sell.get('Buy', 0),
        buy_sell.get('Sell', 0),
        success_fail.get('Success', 0),
        success_fail.get('Fail', 0),
        sum_pips
        ]
}, index=["Trades","Buy", "Sell", "Success", "Fail", "Pips Earn/Lost"])

print(combined_df)



                 Mar
Trades          30.0
Buy             15.0
Sell            15.0
Success         12.0
Fail            18.0
Pips Earn/Lost  -6.0


In [60]:
trades_df[["time",'Trade','Trade_Success','Pips']].head()

,time,Trade,Trade_Success,Pips
593,2024-03-05 01:30:00,Sell,Fail,-1.0
762,2024-03-05 15:35:00,Buy,Success,1.0
879,2024-03-06 01:20:00,Sell,Fail,-1.0
955,2024-03-06 07:40:00,Buy,Success,1.0
1251,2024-03-07 08:20:00,Sell,Fail,-1.0


#### Functions


In [47]:
def get_month_dates(month_name, year):
    """
    Returns the start and end dates of a given month and year.

    Parameters:
        month_name (str): The abbreviated name of the month (e.g., "Jan", "Feb").
        year (int): The year (e.g., 2024).

    Returns:
        tuple: A tuple containing the start and end dates as datetime objects.
    """
    # Convert month name to month number
    month_number = datetime.strptime(month_name, "%b").month
    
    # Get the first day of the month
    start_date = datetime(year, month_number, 1)
    
    # Get the last day of the month
    last_day = calendar.monthrange(year, month_number)[1]
    end_date = datetime(year, month_number, last_day)
    
    return start_date, end_date


In [53]:
def moving_average_backtest(symbol, timeframe, month, year, num1, num2):
    """
    Performs a moving average crossover backtest for a given symbol, timeframe, and date range.

    Parameters:
        symbol (str): The trading symbol (e.g., "EURUSDm").
        timeframe (int): The MetaTrader 5 timeframe constant (e.g., mt5.TIMEFRAME_M15).
        month (str): The abbreviated name of the month (e.g., "Jan", "Feb").
        year (int): The year (e.g., 2024).
        num1 (int): The period for the first moving average.
        num2 (int): The period for the second moving average.

    Returns:
        dict: A dictionary summarizing trade statistics, including total trades, 
              the number of buy and sell trades, successful and failed trades, 
              and the total pips earned or lost.
    """
    # Get the dates
    start_date = get_month_dates(month, year)[0]
    end_date = get_month_dates(month, year)[1]

    # Retrieve currency data in 15 minute timeframe with a time range
    ticks = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    data = pd.DataFrame(ticks)
    data['time'] = pd.to_datetime(data['time'], unit='s')  # convert time including seconds
    data.rename(columns={"tick_volume": "volume"}, inplace=True)

    # Get the "point" or tick of the pair for symbol presumably = 0.00001
    sym_point = mt5.symbol_info(symbol).point

    #TP AND SP amount
    tp_amount = 10
    sp_amount = 10
    
    # Define pip values and spread adjustment, a point is 0.00001
    take_profit_pips = tp_amount * (sym_point * 100)  # 10 pips, 5$ for take profit
    stop_loss_pips = sp_amount * (sym_point * 100)  # 10 pips, 1$ below entry

    
    # Create column name from num1 and num2
    name1 = "MA_" + str(num1)
    name2 = "MA_" + str(num2)

    # Sample DataFrame with close prices and moving averages (data)
    # Calculate Moving Averages
    data[name1] = talib.SMA(data['close'], timeperiod=num1)
    data[name2] = talib.SMA(data['close'], timeperiod=num2)

    # Initialize columns for trade entry prices, targets, and results
    data['Entry_Price'] = None
    data['Take_Profit'] = None
    data['Stop_Loss'] = None
    data['Trade'] = None  # 'Buy' or 'Sell'
    data['Trade_Success'] = None
    data['Pips'] = None

    # Identify crossovers and calculate thresholds in one pass
    buy_signals = (data[name1] > data[name2]) & (data[name1].shift(1) <= data[name2].shift(1))
    sell_signals = (data[name1] < data[name2]) & (data[name1].shift(1) >= data[name2].shift(1))

    # Buy signal: entry price + half spread, set TP and SL
    data.loc[buy_signals, 'Entry_Price'] = data['close'] + (data['spread']*sym_point)
    data.loc[buy_signals, 'Take_Profit'] = data['Entry_Price'] + take_profit_pips
    data.loc[buy_signals, 'Stop_Loss'] = data['Entry_Price'] - stop_loss_pips
    data.loc[buy_signals, 'Trade'] = 'Buy'

    # Sell signal: entry price - spread, set TP and SL
    data.loc[sell_signals, 'Entry_Price'] = data['close'] - (data['spread']*sym_point)
    data.loc[sell_signals, 'Take_Profit'] = data['Entry_Price'] - take_profit_pips
    data.loc[sell_signals, 'Stop_Loss'] = data['Entry_Price'] + stop_loss_pips
    data.loc[sell_signals, 'Trade'] = 'Sell'

    # Check if TP or SL is hit in the next 1 to 200 periods for each trade row
    for i, row in data.iterrows():
        if row['Trade'] in ['Buy', 'Sell']:
            trade_success = 'No Result'
            pips_result = 0  # Default pip result if no TP or SL is hit

            # Iterate through increasing periods (1 to 200) to check TP/SL
            for period in range(1, 1000):
                # Define the future high and low within the current period
                future_high = data['high'][i:i+period].max()
                future_low = data['low'][i:i+period].min()

                # Set conditions based on trade type
                if row['Trade'] == 'Buy':
                    take_profit_hit = future_high >= row['Take_Profit']
                    stop_loss_hit = future_low <= row['Stop_Loss']
                else:  # Sell
                    take_profit_hit = future_low <= row['Take_Profit']
                    stop_loss_hit = future_high >= row['Stop_Loss']

                # Determine if TP or SL is hit within this period
                if take_profit_hit:
                    trade_success = 'Success'
                    pips_result = tp_amount
                    break  # Exit the loop once TP is hit
                elif stop_loss_hit:
                    trade_success = 'Fail'
                    pips_result = -1 * sp_amount
                    break  # Exit the loop once SL is hit

            # Update the DataFrame with the trade result
            data.at[i, 'Trade_Success'] = trade_success
            data.at[i, 'Pips'] = pips_result

    # Filter and view only the trades
    trades_df = data.dropna(subset=['Trade'])

    # Get the month name
    month_name = month

    # Extract counts for each category
    buy_sell = trades_df["Trade"].value_counts()
    success_fail = trades_df["Trade_Success"].value_counts()
    total_trades = trades_df.shape[0]   # For the total count of trades
    sum_pips = trades_df.groupby("Trade_Success")["Pips"].sum().sum() # Sum of pips

    combined_dict = {
            "Trades": total_trades,
            "Buy": buy_sell.get('Buy', 0),
            "Sell": buy_sell.get('Sell', 0),
            "Success": success_fail.get('Success', 0),
            "Fail": success_fail.get('Fail', 0),
            "No Result": success_fail.get('No Result', 0),
            "Pips Earn/Lost": sum_pips
        }
    

    return combined_dict


In [49]:
def backtest_multiple_months(symbol, timeframe, year, num1, num2):
    """
    Runs the moving average backtest for multiple months of a given year.

    Parameters:
        symbol (str): The trading symbol (e.g., "EURUSDm").
        timeframe (int): The MetaTrader 5 timeframe constant (e.g., mt5.TIMEFRAME_M15).
        year (int): The year for the backtest (e.g., 2024).
        num1 (int): The period for the first moving average.
        num2 (int): The period for the second moving average.

    Returns:
        pd.DataFrame: A transposed DataFrame where rows are metrics and columns are months.
    """
    months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
    data = []

    for month in months:
        result = moving_average_backtest(symbol, timeframe, month, year, num1, num2)
        data.append(result)

    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(data, index=months)
    df = df.transpose()  # Transpose to make months columns and metrics rows
    return df


In [56]:
def best_moving_averages(symbol, timeframe, year):
    """
    Backtest multiple moving average crossovers strategies for a given symbol, timeframe, and year.
    
    Args:
        symbol (str): The trading symbol (e.g., 'EURUSDm').
        timeframe: Timeframe for the backtest (e.g., mt5.TIMEFRAME_H2).
        year (int): Year for the backtest.
    
    FYI:
        This function will take some time and some memory power to work
    
    Returns:
        pd.DataFrame: A sorted DataFrame of the results.
    """
    # Define the ranges for short and long moving averages
    ma_short_range = range(2, 50)
    ma_long_range = range(2, 50)
    
    # Initialize an empty list to store results
    results = []

    for long in ma_long_range:
        for short in ma_short_range:
            if short >= long:  # Ensure short MA is less than long MA
                continue
            
            bmm = backtest_multiple_months(symbol, timeframe, year, short, long)
            total_trade = bmm.iloc[0].sum()
            buy_trades = bmm.iloc[1].sum()
            sell_trades = bmm.iloc[2].sum()
            successful_trades = bmm.iloc[3].sum()
            failed_trades = bmm.iloc[4].sum()
            no_result_trades = bmm.iloc[5].sum()
            pip_sum = bmm.iloc[6].sum()
            if successful_trades > 0:
                yr_win_rate = int((successful_trades / total_trade) * 100)
            else:
                yr_win_rate = 0
            
            # Append the result as a dictionary
            results.append({
                "Symbol": symbol,
                "Year": year,
                "Short MA": short,
                "Long MA": long,
                "Total Trades": total_trade,
                "Successful Trades": successful_trades,
                "Pips Earned/Lost": pip_sum,
                "Win Rate (%)": yr_win_rate
            })

    # Convert the list of results to a DataFrame
    results_df = pd.DataFrame(results)

    # Sort the DataFrame by 'Win Rate (%)' and 'Pips Earned/Lost' in descending order
    sorted_results_df = results_df.sort_values(
        by=["Win Rate (%)", "Pips Earned/Lost"],
        ascending=[False, False]
    )
    
    return sorted_results_df


In [55]:
symbol = "EURUSDm"
timeframe = mt5.TIMEFRAME_H2
bma_2024= best_moving_averages(symbol, timeframe, 2024)
bma_2024

,Symbol,Year,Short MA,Long MA,Total Trades,Successful Trades,Pips Earned/Lost,Win Rate (%)
1,EURUSDm,2024,2,4,770,241,10,31
0,EURUSDm,2024,2,3,1111,343,-30,30
2,EURUSDm,2024,3,4,890,270,-120,30


In [40]:
symbol = "EURUSDm"
timeframe = mt5.TIMEFRAME_H1

x= backtest_multiple_months(symbol,timeframe,2024,2,3)
x

,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
Trades,200,175,187,188,196,176,204,208,182,201,179,123
Buy,100,87,93,94,98,88,102,104,91,100,90,62
Sell,100,88,94,94,98,88,102,104,91,101,89,61
Success,92,83,83,88,92,75,88,97,94,97,80,58
Fail,100,85,95,93,99,90,102,106,84,99,93,61
No Result,8,7,9,7,5,11,14,5,4,5,6,4
Pips Earn/Lost,-24,-6,-36,-15,-21,-45,-42,-27,30,-6,-39,-9


__Info__
- Now we need to find the moving average pairs with the best average win rate (%) across the years

##### Testing different timeframes

H2

In [ ]:
symbol = "EURUSDm"
timeframe = mt5.TIMEFRAME_H2

# List of years to process
years = range(2019, 2025)

# Initialize an empty list to store results for each year
results = []

# Track progress with tqdm
for year in tqdm(years, desc="Processing Years", unit="year"):
    result = best_moving_averages(symbol, timeframe, year)
    results.append(result)

# Combine all dataframes into one
combined_results = pd.concat(results)

# Group by 'Short MA' and 'Long MA' and calculate the average win rate and total trades
average_win_rates = combined_results.groupby(['Short MA', 'Long MA']).agg(
    {'Win Rate (%)': 'mean', 'Total Trades': 'mean', "Failed Trades": 'mean', "Pips Earned/Lost": 'mean'}
).reset_index()

# Rename columns for clarity if needed
average_win_rates.rename(columns={
    'Win Rate (%)': 'Avg Yearly Win Rate (%)',
    'Total Trades': 'Avg Yearly Total Trades',
    "Failed Trades": 'Avg Yearly Failed Trades',
    "Pips Earned/Lost": "Average Pips Earned/Lost"
}, inplace=True)

# Sort by 'Average Win Rate (%)' in descending order
best_pairs_h2 = average_win_rates.sort_values(by='Avg Yearly Win Rate (%)', ascending=False)

# Convert all columns in the dataframe to integers
best_pairs_h2 = best_pairs_h2.astype(int)

best_pairs_h2.head()


For H1

In [11]:
symbol = "EURUSDm"
timeframe = mt5.TIMEFRAME_H1

# List of years to process
years = range(2014, 2025)

# Initialize an empty list to store results for each year
results = []

# Track progress with tqdm
for year in tqdm(years, desc="Processing Years", unit="year"):
    result = best_moving_averages(symbol, timeframe, year)
    results.append(result)

# Combine all dataframes into one
combined_results = pd.concat(results)

# Group by 'Short MA' and 'Long MA' and calculate the average win rate and total trades
average_win_rates = combined_results.groupby(['Short MA', 'Long MA']).agg(
    {'Win Rate (%)': 'mean', 'Total Trades': 'mean', "Failed Trades": 'mean', "Pips Earned/Lost": 'mean'}
).reset_index()

# Rename columns for clarity if needed
average_win_rates.rename(columns={
    'Win Rate (%)': 'Avg Yearly Win Rate (%)',
    'Total Trades': 'Avg Yearly Total Trades',
    "Failed Trades": 'Avg Yearly Failed Trades',
    "Pips Earned/Lost": "Average Pips Earned/Lost"
}, inplace=True)

# Sort by 'Average Win Rate (%)' in descending order
best_pairs_h1 = average_win_rates.sort_values(by='Avg Yearly Win Rate (%)', ascending=False)

# Convert all columns in the dataframe to integers
best_pairs_h1 = best_pairs_h1.astype(int)

best_pairs_h1.head()


Processing Years: 100%|██████████| 11/11 [1:20:39<00:00, 439.94s/year]


,Short MA,Long MA,Avg Yearly Win Rate (%),Avg Yearly Total Trades,Avg Yearly Failed Trades,Average Pips Earned/Lost
803,24,25,62,425,218,819
723,21,23,60,331,177,593
663,19,20,59,499,270,873
664,19,21,59,350,191,604
750,22,23,59,448,236,824


__Info__
- We have 252 trading days in a year (This basically removes weekends and certaion holidays).

_Remember_
- MT5 has a data retrieval limit of 50,000 records, starting from the current date and time. The amount of historical data you can access in terms of years depends on the timeframe you select. Shorter timeframes (e.g., 1-minute) will cover fewer years, while longer timeframes (e.g., 1-day) will cover more years within the 50,000-record limit.
- I'm yet to find a way to bypass this restriction but for now here is the timeframes and data limits
- _Also General data starts from_ ___Jan 2014___
    - 1-minute (M1) ≈ 1.5 months
    - 5-minute (M5) ≈ 8.7 months
    - 15-minute (M15) ≈ 2.2 years
    - 30-minute (M30) ≈ 4.4 years
    - 1-hour (H1) ≈ 8.7 years
    - 2-hour (H2) ≈ 17.4 years
    - 4-hour (H4) ≈ 34.8 years
    - 6-hour (H6) ≈ 52.2 years
    - 8-hour (H8) ≈ 69.6 years
    - 12-hour (H12) ≈ 103.7 years
    - 1-day (D1) ≈ 136.9 years
    - 1-week (W1) ≈ 961.5 years
    - 1-month (MN) ≈ 4166.7 years


##### Function for more data (optional)

In [42]:
from datetime import datetime, timedelta

def extract_data_by_day(symbol, timeframe, start_date, end_date):
    """
    Extract historical data from MetaTrader 5 by iterating through each day
    between start_date and end_date, appending data for each day.

    Parameters:
        symbol (str): The trading symbol (e.g., "EURUSD").
        timeframe (int): The MetaTrader 5 timeframe constant (e.g., mt5.TIMEFRAME_M1).
        start_date (datetime): The start date for data extraction.
        end_date (datetime): The end date for data extraction.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted data.
    """
    all_data = []  # List to store data for each day

    # Start the loop from start_date to end_date
    current_date = start_date
    while current_date <= end_date:
        next_day = current_date + timedelta(days=1)  # Move to the next day
        
        # Fetch data for the current day
        day_data = mt5.copy_rates_range(symbol, timeframe, current_date, next_day)
        
        if day_data is None or len(day_data) == 0:
            pass
        else:
            # Append the data for the current day to the list
            all_data.append(pd.DataFrame(day_data))
        
        # Move to the next day
        current_date = next_day

    # Combine all the data into a single DataFrame
    if all_data:
        final_data = pd.concat(all_data, ignore_index=True)
        final_data['time'] = pd.to_datetime(final_data['time'], unit='s')
        return final_data
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no data was found



yeah no data was found at certain timeframes at certain years

In [36]:
# Define the timeframe and date range
timeframe = mt5.TIMEFRAME_M10
start_date = datetime(2021, 1, 1)
end_date = datetime(2021, 2, 20)

data = pd.DataFrame(mt5.copy_rates_range("EURUSDm", timeframe, start_date, end_date))

# data = extract_data_by_day("EURUSDm", timeframe, start_date, end_date)
data

,time,open,high,low,close,tick_volume,spread,real_volume
0,1647502800,1.1032,1.10395,1.10297,1.10386,265,8,0


### Charts

In [ ]:
view_trades= trades_df[['time','Trade','Trade_Success']]

In [ ]:
# Specifying the area
data_plot = data.loc[5950:6100].copy()

# Create the figure
fig = go.Figure()

# Add the candlestick trace
fig.add_trace(go.Candlestick(
    x=data_plot['time'],
    open=data_plot['open'],
    high=data_plot['high'],
    low=data_plot['low'],
    close=data_plot['close'],
    name="Candlestick"
))

# Add moving average for MA_16
fig.add_trace(go.Scatter(
    x=data_plot['time'],
    y=data_plot['MA_16'],
    mode='lines',
    line=dict(color='blue', width=1),
    name="MA_16"
))

# Add moving average for MA_50
fig.add_trace(go.Scatter(
    x=data_plot['time'],
    y=data_plot['MA_50'],
    mode='lines',
    line=dict(color='red', width=1),
    name="MA_50"
))

# Update layout for better readability
fig.update_layout(
    title="Candlestick Chart with Moving Averages",
    xaxis_title="Time",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

# Show the figure
fig.show()

# view trades
# print(view_trades)

In [ ]:
data.loc[213:216]

,time,open,high,low,close,volume,spread,real_volume,MA_16,MA_50,Entry_Price,Take_Profit,Stop_Loss,Trade,Trade_Success
213,2024-10-24 17:45:00,1.08230,1.08239,1.08221,1.08223,98,9,0,1.081509,1.080588,None,None,None,None,None
214,2024-10-24 17:50:00,1.08223,1.08226,1.08210,1.08212,105,9,0,1.081596,1.080626,None,None,None,None,None
215,2024-10-24 17:55:00,1.08213,1.08219,1.08190,1.08204,144,9,0,1.081681,1.080668,None,None,None,None,None
216,2024-10-24 18:00:00,1.08202,1.08203,1.08191,1.08201,131,9,0,1.081756,1.080711,None,None,None,None,None


In [63]:
symbol_info= mt5.symbol_info("BTCUSDm")
point = symbol_info.point
point

0.01

In [59]:
mt5.symbol_info(symbol).spread * point

28.8

In [65]:
1 * symbol_info.point * 10

0.1

In [71]:
symbol_info._asdict()

{'custom': False,
 'chart_mode': 0,
 'select': True,
 'visible': True,
 'session_deals': 0,
 'session_buy_orders': 0,
 'session_sell_orders': 0,
 'volume': 0,
 'volumehigh': 0,
 'volumelow': 0,
 'time': 1734250622,
 'digits': 2,
 'spread': 2880,
 'spread_float': True,
 'ticks_bookdepth': 0,
 'trade_calc_mode': 5,
 'trade_mode': 4,
 'start_time': 0,
 'expiration_time': 0,
 'trade_stops_level': 0,
 'trade_freeze_level': 0,
 'trade_exemode': 2,
 'swap_mode': 1,
 'swap_rollover3days': 5,
 'margin_hedged_use_leg': False,
 'expiration_mode': 15,
 'filling_mode': 3,
 'order_mode': 127,
 'order_gtc_mode': 0,
 'option_mode': 0,
 'option_right': 0,
 'bid': 101740.99,
 'bidhigh': 102806.43,
 'bidlow': 101213.18,
 'ask': 101769.79,
 'askhigh': 102839.26,
 'asklow': 101241.98,
 'last': 0.0,
 'lasthigh': 0.0,
 'lastlow': 0.0,
 'volume_real': 0.0,
 'volumehigh_real': 0.0,
 'volumelow_real': 0.0,
 'option_strike': 0.0,
 'point': 0.01,
 'trade_tick_value': 0.01,
 'trade_tick_value_profit': 0.01,
 'trad

### Conclusion

- There were no substantial results within the 2 to 50 range with a 1:1 risk to reward ratio from 300 to 1000 pips